In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

data = pd.read_csv('processed_data/with_ranges_features.csv')

# Determine the number of classes and create a global label encoder
label_encoder = LabelEncoder()
label_encoder.fit(data['HeartRateClass'])
num_classes = len(label_encoder.classes_)

In [50]:
def preprocess(df):
    # Extract features and label
    features = df[['Elevation', 'Distance', 'HeartRateQuotient', 'HeartRateRange', 'Cadence', 'Speed', 'SpeedRange', 'SpeedQuotient']].to_numpy()
    features = features.reshape(features.shape[0], 1, features.shape[1])

    label = df['HeartRateClass'].to_numpy()
    
    # Encode labels as integers and then convert to categorical
    label = label_encoder.transform(label)
    print(label)
    #label = to_categorical(label, num_classes=num_classes)
    
    return train_test_split(features, label, test_size=0.2, random_state=42)
    
x_train, x_test, y_train, y_test = preprocess(data)

[3 3 4 ... 6 6 6]


In [51]:
from tcn import TCN, compiled_tcn

model = keras.Sequential([
    keras.layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
    keras.layers.Dropout(rate=0.2),
    TCN(
        nb_filters=90,
        kernel_size=3,
        dilations=[2**i for i in range(6)],
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(num_classes, activation='softmax')  # Multi-class classification
])

model.summary()
model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_10 (Dropout)        (None, 1, 8)              0         
                                                                 
 tcn_10 (TCN)                (None, 90)                271350    
                                                                 
 flatten_1 (Flatten)         (None, 90)                0         
                                                                 
 dense_10 (Dense)            (None, 10)                910       
                                                                 
Total params: 272,260
Trainable params: 272,260
Non-trainable params: 0
_________________________________________________________________


In [52]:
print(y_train[:256].shape)
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=256)

(256,)
Epoch 1/10
5174/5174 [==============================] - 224s 43ms/step - loss: nan - accuracy: 7.4749e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/10
5174/5174 [==============================] - 223s 43ms/step - loss: nan - accuracy: 2.2651e-06 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/10
5174/5174 [==============================] - 228s 44ms/step - loss: nan - accuracy: 2.2651e-06 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/10
5174/5174 [==============================] - 230s 44ms/step - loss: nan - accuracy: 2.2651e-06 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/10
5174/5174 [==============================] - 226s 44ms/step - loss: nan - accuracy: 2.2651e-06 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/10
5174/5174 [==============================] - 226s 44ms/step - loss: nan - accuracy: 2.2651e-06 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/10
5174/5174 [==============================] - 216s 42ms/step - loss: nan - accuracy: 2.26